In [45]:
%%sql -d standard
SELECT 
  id, title, body, view_count, tags
FROM
  `gcp-samples2.stackoverflow_demo.top100K_posts`
ORDER BY
  view_count DESC
LIMIT
  5

# (for full query on 10M posts takes 45 sec with 17.9GB)

id,title,body,view_count,tags
927358,How to undo last commit(s) in Git?,<p>I committed the wrong files to <strong>Git</strong>.</p> <p>How can I undo that commit?</p>,4077668,git|git-rebase|git-commit|git-reset|git-revert
5585779,Converting String to Int in Java?,"<p>How can I convert a <code>String</code> to an <code>int</code> in Java?</p> <p>My String contains only numbers and I want to return the number it represents.</p> <p>For example, given the string <code>""1234""</code> the result should be the number <code>1234</code>.</p>",3825994,java|string|type-conversion
1789945,How to check if one string contains another substring in JavaScript?,"<p>Usually, I would expect a <code>String.contains()</code> method, but there doesn't seem to be one. What is a reasonable way to check for this?</p>",3427502,javascript|string|substring|string-matching
2003505,How to delete a Git branch both locally and remotely?,"<p>I want to delete a branch both locally and on my remote project fork on <a href=""http://en.wikipedia.org/wiki/GitHub"" rel=""nofollow noreferrer"">GitHub</a>.</p> <h3>Failed Attempts to Delete Remote Branch</h3> <pre><code>$ git branch -d remotes/origin/bugfix error: branch 'remotes/origin/bugfix' not found. $ git branch -d origin/bugfix error: branch 'origin/bugfix' not found. $ git branch -rd origin/bugfix Deleted remote branch origin/bugfix (was 2a14ef7). $ git push Everything up-to-date $ git pull From github.com:gituser/gitproject * [new branch] bugfix -&gt; origin/bugfix Already up-to-date. </code></pre> <p>What do I need to do differently to successfully delete the <code>remotes/origin/bugfix</code> branch both locally and on GitHub?</p>",3416408,git|git-branch|git-remote
503093,How do I redirect to another page in jQuery?,<p>How can I redirect the user from one page to another using jQuery?</p>,3382579,javascript|jquery|redirect


In [48]:
%%sql -d standard
CREATE TEMPORARY FUNCTION segmentation(body STRING)
RETURNS ARRAY<STRING>
LANGUAGE js AS """

// remove HTML tags, convert to lowercase and extract words
return body.replace(/<[^>]+>/g, '').toLowerCase().match(/\\w\\w+/g);

""";

SELECT
  id, 
  segmentation(CONCAT(title, body, tags)) as words
FROM
  `gcp-samples2.stackoverflow_demo.top100K_posts`
LIMIT
  5

# (for full query on 10M posts takes about 30 secs with 14.1GB)

id,words
1789945,"['how', 'to', 'check', 'if', 'one', 'string', 'contains', 'another', 'substring', 'in', 'javascript', 'usually', 'would', 'expect', 'string', 'contains', 'method', 'but', 'there', 'doesn', 'seem', 'to', 'be', 'one', 'what', 'is', 'reasonable', 'way', 'to', 'check', 'for', 'this', 'javascript', 'string', 'substring', 'string', 'matching']"
927358,"['how', 'to', 'undo', 'last', 'commit', 'in', 'git', 'committed', 'the', 'wrong', 'files', 'to', 'git', 'how', 'can', 'undo', 'that', 'commit', 'git', 'git', 'rebase', 'git', 'commit', 'git', 'reset', 'git', 'revert']"
503093,"['how', 'do', 'redirect', 'to', 'another', 'page', 'in', 'jquery', 'how', 'can', 'redirect', 'the', 'user', 'from', 'one', 'page', 'to', 'another', 'using', 'jquery', 'javascript', 'jquery', 'redirect']"
5585779,"['converting', 'string', 'to', 'int', 'in', 'java', 'how', 'can', 'convert', 'string', 'to', 'an', 'int', 'in', 'java', 'my', 'string', 'contains', 'only', 'numbers', 'and', 'want', 'to', 'return', 'the', 'number', 'it', 'represents', 'for', 'example', 'given', 'the', 'string', '1234', 'the', 'result', 'should', 'be', 'the', 'number', '1234', 'java', 'string', 'type', 'conversion']"
2003505,"['how', 'to', 'delete', 'git', 'branch', 'both', 'locally', 'and', 'remotely', 'want', 'to', 'delete', 'branch', 'both', 'locally', 'and', 'on', 'my', 'remote', 'project', 'fork', 'on', 'github', 'failed', 'attempts', 'to', 'delete', 'remote', 'branch', 'git', 'branch', 'remotes', 'origin', 'bugfix', 'error', 'branch', 'remotes', 'origin', 'bugfix', 'not', 'found', 'git', 'branch', 'origin', 'bugfix', 'error', 'branch', 'origin', 'bugfix', 'not', 'found', 'git', 'branch', 'rd', 'origin', 'bugfix', 'deleted', 'remote', 'branch', 'origin', 'bugfix', 'was', '2a14ef7', 'git', 'push', 'everything', 'up', 'to', 'date', 'git', 'pull', 'from', 'github', 'com', 'gituser', 'gitproject', 'new', 'branch', 'bugfix', 'gt', 'origin', 'bugfix', 'already', 'up', 'to', 'date', 'what', 'do', 'need', 'to', 'do', 'differently', 'to', 'successfully', 'delete', 'the', 'remotes', 'origin', 'bugfix', 'branch', 'both', 'locally', 'and', 'on', 'github', 'git', 'git', 'branch', 'git', 'remote']"


In [49]:
%%sql -d standard
SELECT
  word,
  COUNT(distinct id) AS posts
FROM 
  `gcp-samples2.stackoverflow_demo.top100K_posts_segmented`
CROSS JOIN
  UNNEST(words) AS word
GROUP BY
  word
ORDER BY
  posts DESC
LIMIT
  5

# (for full query on 100K posts takes 35 secs with 12.2GB)

word,posts
to,89983
the,86122
in,72679
is,69707
and,64563


```word_dict_0.js
words = [
{"word":"to","posts":"8361"},
{"word":"the","posts":"8260"},
{"word":"is","posts":"6534"},
...
{"word":"ecx","posts":"1"},
{"word":"12t08","posts":"1"},
{"word":"coproductgroup2","posts":"1"}];

word_dict = new Object();
for (var i = 0; i < words.length; i++) {
    word_dict[words[i].word] = words[i].posts;
}
```

In [51]:
%%sql -d standard
CREATE TEMPORARY FUNCTION calc_tf_idf(words ARRAY<STRING>)
RETURNS STRING
LANGUAGE js AS """

// count each word in this post
var word_count = new Object();
for (word of words) {
  if (word_count[word]) {
    word_count[word]++;
  } else {
    word_count[word] = 1;
  }
}

// calculate TF-IDF values for each word
// tf = word count / total word count
// idf = log (100K posts / # of posts with the word)
// tf_idf = tf * idf
var total_posts = 100000;
var words_in_post = words.length;
var tf_idf = new Object();
var norm_sum = 0;
for (word in word_count) {
  if (word_dict[word] == null) {
    word_dict[word] = 1;
  }
  var tf = (word_count[word] / words_in_post);
  var idf = Math.log(total_posts / word_dict[word]);
  tf_idf[word] = tf * idf;
  norm_sum += tf_idf[word]^2;
}

// normarizing TF-IDF values with L2 norm
for (word in tf_idf) {
  tf_idf[word] = (tf_idf[word] / Math.sqrt(norm_sum)).toFixed(5);
}

return JSON.stringify(tf_idf);

"""
OPTIONS (
  library="gs://gcp-samples2-stackoverflow/word_dict_0.js",
  library="gs://gcp-samples2-stackoverflow/word_dict_1.js",
  library="gs://gcp-samples2-stackoverflow/word_dict_2.js",
  library="gs://gcp-samples2-stackoverflow/word_dict_3.js",
  library="gs://gcp-samples2-stackoverflow/word_dict_4.js",
  library="gs://gcp-samples2-stackoverflow/word_dict_5.js"
);

SELECT
  id, calc_tf_idf(words) AS tf_idf
FROM
  `gcp-samples2.stackoverflow_demo.top100K_posts_segmented` AS posts
LIMIT
  5

# (for full query on 10M posts takes 70 secs with 12.2GB)

id,tf_idf
7172784,"{""17"":""0.00114"",""24"":""0.00118"",""30"":""0.00207"",""50"":""0.00103"",""415"":""0.00365"",""1051"":""0.00462"",""2011"":""0.00121"",""8080"":""0.00138"",""how"":""0.00017"",""to"":""0.00013"",""post"":""0.00402"",""json"":""0.00362"",""data"":""0.00158"",""with"":""0.00068"",""curl"":""0.00963"",""from"":""0.00029"",""terminal"":""0.00123"",""commandline"":""0.00178"",""test"":""0.00213"",""spring"":""0.00349"",""rest"":""0.00337"",""use"":""0.00074"",""ubuntu"":""0.00116"",""and"":""0.00033"",""installed"":""0.00087"",""on"":""0.00025"",""it"":""0.00044"",""want"":""0.00068"",""my"":""0.00075"",""application"":""0.00121"",""wrote"":""0.00121"",""code"":""0.00071"",""at"":""0.00049"",""java"":""0.00117"",""side"":""0.00203"",""however"":""0.00065"",""am"":""0.00035"",""trying"":""0.00048"",""an"":""0.00028"",""example"":""0.00053"",""is"":""0.00036"",""like"":""0.00032"",""this"":""0.00083"",""value"":""0.00104"",""type"":""0.00216"",""tip"":""0.00337"",""targetmodule"":""0.00577"",""target"":""0.00206"",""configurationgroup"":""0.00577"",""null"":""0.00291"",""name"":""0.00104"",""configuration"":""0.00873"",""deneme"":""0.01015"",""description"":""0.00325"",""identity"":""0.00269"",""version"":""0.00141"",""systemid"":""0.00508"",""active"":""0.00235"",""true"":""0.00132"",""command"":""0.00143"",""accept"":""0.00118"",""http"":""0.00224"",""method"":""0.00177"",""override"":""0.00100"",""put"":""0.00325"",""localhost"":""0.00099"",""xx"":""0.00154"",""xxx"":""0.00132"",""xxxx"":""0.00160"",""returns"":""0.00087"",""error"":""0.00137"",""unsupported"":""0.00166"",""media"":""0.00129"",""server"":""0.00115"",""apache"":""0.00099"",""coyote"":""0.00164"",""content"":""0.00149"",""text"":""0.00054"",""html"":""0.00059"",""charset"":""0.00119"",""utf"":""0.00096"",""length"":""0.00089"",""date"":""0.00081"",""wed"":""0.00168"",""aug"":""0.00169"",""08"":""0.00128"",""gmt"":""0.00149"",""the"":""0.00026"",""refused"":""0.00160"",""request"":""0.00167"",""because"":""0.00067"",""entity"":""0.00115"",""in"":""0.00008"",""format"":""0.00168"",""not"":""0.00026"",""supported"":""0.00125"",""by"":""0.00046"",""requested"":""0.00261"",""resource"":""0.00109"",""for"":""0.00019"",""tomcat"":""0.00123"",""log"":""0.00083"",""ui"":""0.00103"",""webapp"":""0.00149"",""conf"":""0.00138"",""clear"":""0.00098"",""any"":""0.00036"",""ideas"":""0.00088"",""about"":""0.00062"",""right"":""0.00069"",""of"":""0.00015"",""edit"":""0.00069"",""have"":""0.00019"",""tested"":""0.00132"",""get"":""0.00033"",""delete"":""0.00096"",""they"":""0.00069"",""work"":""0.00050"",""requestmapping"":""0.00155"",""requestmethod"":""0.00162"",""public"":""0.00063"",""updateconfiguration"":""0.00254"",""httpservletresponse"":""0.00332"",""response"":""0.00188"",""requestbody"":""0.00197"",""consider"":""0.00129"",""valid"":""0.00105"",""tag"":""0.00097"",""setname"":""0.00172"",""worked"":""0.00105"",""todo"":""0.00134"",""if"":""0.00029"",""occurs"":""0.00127"",""senderror"":""0.00261"",""sc_not_found"":""0.00289"",""return"":""0.00057"",""mvc"":""0.00103""}"
953918,"{""how"":""0.00180"",""to"":""0.00114"",""align"":""0.01234"",""the"":""0.00242"",""middle"":""0.02801"",""horizontally"":""0.01613"",""width"":""0.02588"",""of"":""0.00325"",""pagei"":""0.01700"",""have"":""0.00209"",""div"":""0.01583"",""tag"":""0.01047"",""with"":""0.00246"",""set"":""0.00567"",""800px"":""0.03974"",""when"":""0.00371"",""browser"":""0.00948"",""is"":""0.00098"",""greater"":""0.01551"",""than"":""0.00798"",""it"":""0.00471"",""shouldn"":""0.01415"",""stretch"":""0.01731"",""but"":""0.00202"",""should"":""0.00582"",""bring"":""0.01650"",""page"":""0.00684"",""css"":""0.00832"",""html"":""0.00637"",""alignment"":""0.01820"",""center"":""0.01136""}"
40471,"{""differences"":""0.06406"",""between"":""0.03702"",""hashmap"":""0.11006"",""and"":""0.00561"",""hashtable"":""0.13619"",""what"":""0.00771"",""are"":""0.01037"",""the"":""0.00096"",""in"":""0.00205"",""j

In [54]:
%%sql -d standard
CREATE TEMPORARY FUNCTION calc_similarity(tf_idf_json_0 STRING, tf_idf_json_1 STRING)
RETURNS FLOAT64
LANGUAGE js AS """

// parse JSON to extract tf_idf
var tf_idf_0 = JSON.parse(tf_idf_json_0);
var tf_idf_1 = JSON.parse(tf_idf_json_1);

// calculate similarity
var similarity = 0;
for (word in tf_idf_0) {
  var t0 = tf_idf_0[word] ? Number(tf_idf_0[word]) : 0;
  var t1 = tf_idf_1[word] ? Number(tf_idf_1[word]) : 0;
  similarity += t0 * t1;
}

return similarity;
""";

SELECT
  title,
  body,
  tags,
  similarity
FROM
  (
    SELECT
      t1.id, 
      calc_similarity(tf_idf_0, t1.tf_idf) AS similarity
    FROM
      (
        SELECT tf_idf AS tf_idf_0
        FROM `gcp-samples2.stackoverflow_demo.top100K_posts_tf_idf` AS t0
        WHERE id = 5585779
      )
    CROSS JOIN
      `gcp-samples2.stackoverflow_demo.top100K_posts_tf_idf` AS t1
    ORDER BY
      similarity DESC
    LIMIT
      10
  )
JOIN
  `gcp-samples2.stackoverflow_demo.top100K_posts` AS t2
USING (id)  
ORDER BY
  similarity DESC

# (for full query on 10M posts takes about 80 secs with 31 GB)

title,body,tags,similarity
How to convert string to string[]?,<p>How to convert <code>string</code> type to <code>string[]</code> type in C#?</p>,c#,0.0056626574
How to convert CharSequence to String in Java?,<p>How can I convert a Java <code>CharSequence</code> to a <code>String</code>?</p>,java|string|charsequence,0.004840766
Convert int to string?,<p>How can I convert an <code>int</code> datatype into a <code>string</code> datatype in C#?</p>,c#|string|int,0.0047671244
How to convert a String to CharSequence?,<p>How to convert a String to CharSequence in Java?</p>,java|string|charsequence,0.0047135428
Convert String to SecureString,<p>How to convert String to SecureString?</p>,c#|securestring,0.0046640136
Converting String to Int in Java?,"<p>How can I convert a <code>String</code> to an <code>int</code> in Java?</p> <p>My String contains only numbers and I want to return the number it represents.</p> <p>For example, given the string <code>""1234""</code> the result should be the number <code>1234</code>.</p>",java|string|type-conversion,0.0044056437
How can I convert a long to int in Java?,<p>How can I convert a long to int in Java?</p>,java,0.0042378967
Conversion from exponential form to decimal in Java,<p>I want to convert exponential to decimal. e.g. <code>1.234E3</code> to <code>1234</code>.</p>,java,0.0041916097
C++ String Length?,<p>How should I get the number of characters in a string in C++?</p>,c++|string|string-length,0.0040086603
Converting String array to java.util.List,<p>How do i convert <code>String</code> array to <code>java.util.List</code>?</p>,java|arrays|collections,0.0038235471


In [50]:
#
# A script for generating JS files for loading word_dict.
#
# Note: you need to export 'top100K_posts_word_dict.csv' file on BQ UI beforehand.

# loading word_dict CSV file
import datalab.storage as storage
bucket = storage.Bucket('gcp-samples2-stackoverflow')
word_dict_file = bucket.item('top100K_posts_word_dict.csv') 
word_dict_csv = word_dict_file.read_from().split();
print 'Loaded ' + str(len(word_dict_csv)) + ' words.'

# prepare header and footer for each JS file
file_header = 'words_str = "'
file_footer = '''";
words = words_str.split(" ");
for (var i = 0; i < words.length; i++) {
  word_and_posts = words[i].split(",");
  word_dict[word_and_posts[0]] = word_and_posts[1];
}
'''
file = bucket.item('word_dict_0.js')
file.write_to('word_dict = new Object();\n', 'text/plain')

# create JS file for each 100K words
lines = 0
files = 1
buf = ''
for word in word_dict_csv:
  if not (lines == 0 and files == 0):
    buf += word + ' ' # skipping the CSV header
  if lines == 50000:
    file = bucket.item('word_dict_' + str(files) + '.js')
    file.write_to(file_header + buf + file_footer, 'text/plain')
    lines = 0
    files += 1
    buf = ''
  lines += 1

print 'Created ' + str(files) + ' files.'

Loaded 270587 words.
Created 6 files.
